In [13]:
import os
from subprocess import check_call
import yaml

import intake

import operators as ops
import utils

In [2]:
esm_collection_json = 'data/campaign-cesm2-cmip6-timeseries.json'
catalog = intake.open_esm_datastore(esm_collection_json)
catalog

,unique
Unnamed: 0,312503
experiment,13
case,33
component,6
stream,22
variable,2641
date_range,596
member_id,12
path,312503
ctrl_branch_year,18


In [3]:
with open('datasets.yml') as fid:
    datasets = yaml.safe_load(fid)

In [4]:
data_dir = f'/glade/scratch/{os.environ["USER"]}/cesm2-marbl-data'
if True:
    check_call(['rm', '-fr', data_dir])
os.makedirs(data_dir, exist_ok=True)        

In [5]:
with open('_config_calc.yml') as fid:
    config_dict = yaml.load(fid, Loader=yaml.Loader)

_collections = config_dict['data_collections']



In [8]:
cluster, client = utils.get_ClusterClient()
cluster.scale(12) #adapt(minimum_jobs=0, maximum_jobs=24)
client

Client Scheduler: tcp://10.12.206.62:42593 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
clobber = False

kwargs = {'cdf_kwargs': dict(chunks={'time': 12}, decode_coords=False)}

dfs = {}; paths = {}
for collection_type, exp_dict in datasets.items():
    if collection_type != 'epoch_mean':
        continue
    for experiment, variable_list in exp_dict.items():
            
            cat = catalog.search(
                experiment=experiment, 
                variable=variable_list, 
                stream='pop.h')
            
            if collection_type == 'timeseries':            
                for path in cat.df.path:
                    data_vol_GB[key] += os.stat(path).st_size / 1024**3
                    check_call(['ln', '-sf', path, data_dir+'/.'])
            
            elif collection_type == 'epoch_mean':
                time_range = _collections['epoch_mean']['experiment'][experiment]
                sel_dict = dict(time=slice(time_range[0], time_range[1]))
                for v in variable_list:
                    cat_sub = cat.search(variable=v)
                    keys = list(cat_sub.keys())
                    assert len(keys) == 1
                    
                    name = keys[0].replace('ocn.', 'cesm2-cmip6.')
                    name = '.'.join([name, v, '-'.join(time_range)])
                    file_out = f'{data_dir}/{name}.zarr'
                    if os.path.exists(file_out) and not clobber:
                        continue 
                    dsets = cat_sub.to_dataset_dict(**kwargs)
                    _, ds = dsets.popitem()
                    ds = ops.center_time(ds)
                    ds = ds.sel(**sel_dict)[[v]]
                    
                    ds.to_zarr(file_out, mode='w')


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream'


In [ ]:
# data_dir = f'/glade/scratch/{os.environ["USER"]}/cesm2-marbl-data'
# os.makedirs(data_dir, exist_ok=True)

# data_vol_GB = {}
# for key, paths in paths.items():
#     data_vol_GB[key] = 0.
#     for path in paths[key]:
#         data_vol_GB[key] += os.stat(path).st_size / 1024**3
#         check_call(['ln', '-sf', path, data_dir+'/.'])
        
#     print(f'Data volume {key}: {data_vol_GB[key]:0.2f} GB')    

In [16]:
300/12

25.0